# SB3 を用いた 2 リンクアームの躍度最小化軌道の学習(https://github.com/DLR-RM/stable-baselines3)
### この jupyternotebook は以下のような構造になっています
1. モジュール管理
2. パラメータ管理
3. 環境設定
4. 環境が正しく動くかの確認
5. エピソードごとの報酬などの学習内部指標を定期的に保存するためのクラス
6. 学習
7. 学習済みデータを用いたシミュレーション
8. ベストモデルを用いたシミュレーション
9. エピソードごとの様々な報酬の推移などの多指標をプロット

## モジュール管理


In [ ]:
# ===== 基本的なライブラリ =====
import os             # ファイルパスやディレクトリの操作（例：ログ保存先の作成など）
import json           # 設定ファイルや学習結果をJSON形式で保存・読み込むために使用
import csv            # 結果やログをCSV形式で出力するために使用
import time           # 実行時間の計測やスリープ処理などに利用
import glob           # 特定パターンのファイル探索（例："*.csv" など）
from typing import Optional, List  # 型ヒントのため（関数の引数・戻り値の明示）
from datetime import datetime      # 実行開始時刻やログのタイムスタンプ記録に利用
import numpy as np    # 数値計算用ライブラリ（ベクトル・行列演算、乱数生成など）
import pandas as pd   # データ処理やCSVの読み書き・集計・可視化に便利

# ===== 可視化関連 =====
import matplotlib.pyplot as plt  # 学習曲線、報酬推移、動作ログなどの可視化に使用

# ===== 強化学習関連（Stable-Baselines3） =====
from stable_baselines3 import PPO, SAC, TD3, DDPG, A2C  
# → 代表的な強化学習アルゴリズム（方策勾配系やアクタークリティック系）をインポート

from stable_baselines3.common.callbacks import EvalCallback, BaseCallback  
# → 学習中に評価や早期終了などを行うためのコールバック機能を利用するため

from stable_baselines3.common.env_checker import check_env  
# → 独自実装したGym環境がSB3に適合しているかを検証するための関数

from collections import deque  # 一定長の履歴（移動平均など）を保持するのに便利

# ===== Gym関連 =====
import gymnasium as gym  # 強化学習環境の作成・管理（GymnasiumはGymの後継ライブラリ）


## パラメータ管理

In [ ]:
# ===== シミュレーションパラメータ =====
DT = 0.01           # シミュレーションの時間刻み (秒)
L = 1.0             # アームの長さ (m)
STEPS_MAX = 200     # 1エピソードあたりの最大ステップ数
THRESHOLD = 0.03    # ゴール判定の許容距離 (m)

# ===== アクションと状態の範囲 =====
ACTION_MIN, ACTION_MAX = -200, 200  # アクション範囲（角速度の下限・上限）[deg/s]
THETA_MIN, THETA_MAX = 0, 180       # 関節角度の下限・上限 [deg]
THETA_INIT = [40,100]                     # 初期角度 [deg]
GOAL_POS = [0.0,0.9]               # ゴール位置 (x, y)

# --- 報酬関連（終端 + shaping） ---
REWARD_P_V = 10.0         # 終端速度ペナルティ係数
REWARD_J = 4e-4          # 終端躍度係数（累積にかける）
REWARD_JE_LIM = 1.0      # 比率表示用（infoに出すだけ）

# --- 目標時間と時間軸の扱い（ソフト制約） ---
T_TARGET = 0.45           # 目標所要時間 [s]
SIGMA_T = 2.0           # 時間ボーナスの許容幅 [s]
R_GOAL = 50.0            # 成功時の基礎ボーナス（十分大きくする）
TIME_COST = 1e-3         # 毎ステップの時間コスト（小）

# --- shaping（毎ステップの小さなガイド） ---
SHAPING_DIST_COEFF = 10.0     # 距離改善に対するステップ報酬倍率
SHAPING_JERK_COEFF = 1e-7    # 瞬時躍度ペナルティ係数（小さく）

# --- 停滞検出（進捗が無いと早めに打ち切る） ---
STALL_WINDOW_S = 0.25        # 停滞ウィンドウ長 [s]
STALL_WINDOW = max(1, int(STALL_WINDOW_S / DT))  # ステップ単位
MIN_PROGRESS_PER_WINDOW = 1e-4   # この期間の進捗がこれ未満なら停滞と判定（m）

# ===== 学習設定 =====
TOTAL_TIMESTEPS = 3000000  # 学習全体での総ステップ数
LEARNING_RATE = 0.0003     # 学習率
TAU = 0.01                 #SACを使う際のαの学習りつ
HID_LAY = 64               #隠れ層のノード数
BUFFER_SIZE = 100000        # リプレイバッファのサイズ
BATCH_SIZE = 256             # ミニバッチサイズ
EVAL_FREQ = 100             # 何ステップごとに評価するか
SAVE_INTERVAL = 1           # 何エピソードごとにモデルを保存するか
RUNNNIG_WINDOW = 10 # 直近何エピソードの平均を計算するか


## 環境設定（2リンクアームのリーチング）

In [ ]:
class TwoJointReachingEnv(gym.Env):
    """
    2関節アームのリーチング環境（拡張版）
    - 直近 100 エピソードの成功率に応じて self.sigma_T を自動で更新（先鋭化/鈍化）します。
    - 変更点・追加:
        * self._success_window : deque を使って直近成功フラグを保持（size = 100）
        * self.episode_count : エピソードカウンタ
        * update_sigma_by_success_rate() : 成功率に応じた sigma_T の更新ロジック
        * step() の末尾でエピソード終了時に success を push -> 必要なら sigma を更新
        * info に sigma 関連のログを追加
    - 注意: この実装はエピソード終了（terminated or truncated）ごとにしか success-window を更新しません。
    """

    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()

        # ---------- 基本パラメータ（グローバル変数前提） ----------
        self.dt = DT  # シミュレーション刻み幅（秒）
        # 全長 L を 2 等分して link 長さに割当て
        self.l1 = float(L) * 0.5
        self.l2 = float(L) * 0.5

        # エピソード最大ステップ、ゴール閾値、ゴール位置
        self.max_steps = STEPS_MAX
        self.goal_threshold = THRESHOLD
        self.goal_pos = np.array(GOAL_POS, dtype=np.float32)

        # --- 報酬 / タスクパラメータ（グローバルから読み取り） ---
        self.T_target = T_TARGET
        self.sigma_T = SIGMA_T   # <-- この値を success-rate に応じて更新する
        self.R_goal = R_GOAL
        self.time_cost = TIME_COST
        self.shaping_dist_coeff = SHAPING_DIST_COEFF
        self.shaping_jerk_coeff = SHAPING_JERK_COEFF
        self.terminal_jerk_coeff = REWARD_J       # 終端で累積手先躍度にかける重み
        self.terminal_vel_coeff = REWARD_P_V      # 終端での手先速度ノルム^2 にかける重み

        # 停滞判定ウィンドウ（ステップ数）と最小進捗量
        self.stall_window = STALL_WINDOW
        self.min_progress_per_window = MIN_PROGRESS_PER_WINDOW

        # ---------- 観測空間（9次元） ----------
        obs_low = np.array([-1.0]*4 + [0.0], dtype=np.float32)
        obs_high = np.array([1.0]*4 + [1.0], dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)

        # ---------- アクション空間（各関節の角速度 rad/s, shape=(2,)） ----------
        self.action_space = gym.spaces.Box(
            low=np.radians(ACTION_MIN),
            high=np.radians(ACTION_MAX),
            shape=(2,),
            dtype=np.float32
        )

        # デバッグ用: 各ステップで両関節が同じ角速度になっていないかを確認するためのカウンタ
        self._same_action_counter = 0
        self._same_action_tol = 1e-8

        # ---------- --- sigma 自動更新に関するハイパラ --- ----------
        # window size: 100 episodes （あなたの指定）
        self._success_window_size = 100
        # deque に成功フラグを入れて管理（1=success, 0=failure）
        self._success_window = deque(maxlen=self._success_window_size)
        self._success_window_size = 100

        # episode counter（reset でエピソード開始時にインクリメントされる想定）
        self.episode_count = 0

        # ここだけ追加：ウィンドウ満杯かつこの interval に達したときだけ更新する
        # 例えば 10 にすれば「100エピソードが溜まった後、10エピソードごとに判定」
        self.sigma_update_interval = 10
        self._sigma_update_counter = 0

        # しきい値: この成功率を超えたら "先鋭化（sigma を小さく）" する
        self.sigma_sharpen_threshold = 0.80    # 例: 80% 以上なら先鋭化
        # しきい値: この成功率を下回ったら "鈍化（sigma を大きく）" する
        self.sigma_blunt_threshold = 0.40      # 例: 40% 以下なら鈍化

        # 変化率: 先鋭化するときの乗算ファクタ (<1 で sigma が小さくなる)
        self.sigma_sharpen_factor = 0.90       # 例: sigma *= 0.90（10%小さく）
        # 変化率: 鈍化するときの乗算ファクタ (>1 で sigma が大きくなる)
        self.sigma_blunt_factor = 1.10         # 例: sigma *= 1.10（10%大きく）

        # sigma の上下限（安定化のため）
        self.sigma_min = 0.2
        self.sigma_max = max(1e-3, SIGMA_T * 10.0)

        # ログ／デバッグ用に直近更新内容を記録
        self._last_sigma_update = {"episode": None, "old_sigma": None, "new_sigma": None, "success_rate": None, "action": None}

        # 環境を初期化（reset を呼ぶ）
        self.reset()

    # ---------- ヘルパー ----------
    def set_penalty_weight(self, new_weight):
        """外部から終端躍度重みを上書き"""
        self.terminal_jerk_coeff = float(new_weight)

    def reached_goal(self, dist_to_goal):
        """距離ベースのゴール判定"""
        return float(dist_to_goal) <= float(self.goal_threshold)

    def forward_kinematics(self, thetas):
        """順運動学: thetas = [th1, th2] (rad) -> hand_pos [x,y] (m)"""
        th1, th2 = float(thetas[0]), float(thetas[1])
        x = self.l1 * np.cos(th1) + self.l2 * np.cos(th1 + th2)
        y = self.l1 * np.sin(th1) + self.l2 * np.sin(th1 + th2)
        return np.array([x, y], dtype=np.float32)

    def _get_obs(self):
        """
        観測を生成して返す（正規化済み）。
        観測ベクトルの順序（呼び出し箇所と合わせてください）:
         [hand_x, hand_y, hand_vx, hand_vy, time_scaled]
        （元の仕様の通りに整形して返します）
        """
        # --- 角度スケール（使っていれば） ---
        theta_min_rad = np.radians(THETA_MIN)
        theta_max_rad = np.radians(THETA_MAX)
        theta_clipped = np.clip(self.theta, theta_min_rad, theta_max_rad)
        theta_scaled = 2.0 * (theta_clipped - theta_min_rad) / max(1e-8, (theta_max_rad - theta_min_rad)) - 1.0
        theta_scaled = np.asarray(theta_scaled).ravel()
        theta_scaled = np.clip(theta_scaled, -1.0, 1.0)

        # --- 関節角速度スケール ---
        vel_scale = np.radians(max(abs(ACTION_MIN), abs(ACTION_MAX)))
        theta_vel_scaled = np.asarray(self.theta_vel / max(1e-8, vel_scale)).ravel()
        theta_vel_scaled = np.clip(theta_vel_scaled, -1.0, 1.0)

        # --- 手先位置スケール (m) ---
        pos_scale = (self.l1 + self.l2)
        hand_pos_scaled = np.asarray(self.hand_pos / max(1e-8, pos_scale)).ravel()
        hand_pos_scaled = np.clip(hand_pos_scaled, -1.0, 1.0)

        # --- 手先速度スケール (m/s) ---
        max_joint_speed_rad = vel_scale
        max_hand_speed = max_joint_speed_rad * pos_scale
        hand_vel_scaled = np.asarray(self.hand_vel / max(1e-8, max_hand_speed)).ravel()
        hand_vel_scaled = np.clip(hand_vel_scaled, -1.0, 1.0)

        # --- 時間スケール ---
        time_scaled = np.array([np.clip(float(self.steps) / max(1, int(self.max_steps)), 0.0, 1.0)], dtype=np.float32)

        # 最終 obs を連結して返す（必要に応じて順序や要素を戻してください）
        obs = np.concatenate([
            hand_pos_scaled.astype(np.float32),     # 2
            hand_vel_scaled.astype(np.float32),     # 2
            time_scaled.astype(np.float32)          # 1
        ])
        return obs

    # ---------- Gymnasium API: reset ----------
    def reset(self, seed=None, options=None):
        """
        初期化:
        - THETA_INIT がスカラー（deg）の場合は両関節に同じ初期角度を入れる
        - iterable（長さ2）の場合は関節ごとに初期角度を設定
        戻り値: (obs, info) を返す（gymnasium 互換）
        """
        super().reset(seed=seed)

        # (epi count は reset を呼ぶ側がエピソードを終えたあとに呼ぶことを想定)
        # ここでは reset が呼ばれる度に episode_count をインクリメントします。
        # 外部の学習ループが env.reset() を使ってエピソードを進める場合、
        # episode_count はエピソード数と同期します。
        self.episode_count += 1

        # THETA_INIT の取り扱い（deg -> rad）
        try:
            th_init_arr = np.asarray(THETA_INIT)
            if th_init_arr.size == 1:
                th_init_arr = np.array([th_init_arr.item(), th_init_arr.item()], dtype=np.float32)
            else:
                th_init_arr = th_init_arr.reshape(-1)[:2].astype(np.float32)
        except Exception:
            th_init_arr = np.array([THETA_INIT, THETA_INIT], dtype=np.float32)

        self.theta = np.radians(th_init_arr).astype(np.float32)
        self.steps = 0
        self.t = 0.0

        # 関節微分量の初期化
        self.theta_vel = np.zeros(2, dtype=np.float32)
        self.theta_acc = np.zeros(2, dtype=np.float32)
        self.theta_jerk = np.zeros(2, dtype=np.float32)

        # 手先の状態（順運動学で初期化）
        self.hand_pos = self.forward_kinematics(self.theta)
        self.hand_vel = np.zeros(2, dtype=np.float32)
        self.hand_acc = np.zeros(2, dtype=np.float32)
        self.hand_jerk = np.zeros(2, dtype=np.float32)

        # 終端評価用の累積手先躍度
        self.jerk_sum = 0.0

        # 進捗監視用
        self.prev_dist = np.linalg.norm(self.hand_pos - self.goal_pos)
        self.dist_window = [self.prev_dist]

        # action 同一カウンタをリセット
        self._same_action_counter = 0

        # Gymnasium 規約に合わせて (obs, info) を返す
        return self._get_obs(), {}

    # ---------- Sigma 更新ロジック ----------
    def update_sigma_by_success_rate(self):
        """
        直近 self._success_window_size (=100) エピソードの成功率に基づいて self.sigma_T を更新する。

        動作:
          - success_rate >= sigma_sharpen_threshold -> 先鋭化（sigma *= sigma_sharpen_factor）
          - success_rate <= sigma_blunt_threshold   -> 鈍化（sigma *= sigma_blunt_factor）
          - そうでなければ変更なし

        安定化:
          - sigma は self.sigma_min ～ self.sigma_max にクリップされる
          - 更新履歴は self._last_sigma_update に格納（デバッグ用）
        """
        if len(self._success_window) < self._success_window_size:
            return False

        success_rate = float(sum(self._success_window)) / float(self._success_window_size)
        old_sigma = float(self.sigma_T)

        update_action = None

        if success_rate >= self.sigma_sharpen_threshold:
            new_sigma = old_sigma * float(self.sigma_sharpen_factor)
            update_action = "sharpen"
        elif success_rate <= self.sigma_blunt_threshold:
            new_sigma = old_sigma * float(self.sigma_blunt_factor)
            update_action = "blunt"
        else:
            new_sigma = old_sigma

        new_sigma = float(np.clip(new_sigma, self.sigma_min, self.sigma_max))
        self.sigma_T = new_sigma

        self._last_sigma_update = {
            "episode": int(self.episode_count),
            "old_sigma": float(old_sigma),
            "new_sigma": float(new_sigma),
            "success_rate": float(success_rate),
            "action": update_action
        }
        return True

    # ---------- Gymnasium API: step ----------
    def step(self, action):
        """
        1 ステップ進める（Gymnasium 互換）:
        入力: action (array-like shape (2,), rad/s)
        戻り: obs, reward, terminated, truncated, info

        追加: エピソード終了時に success-window を更新し、
              100 エピソード分が溜まっていれば sigma_T の更新ロジックを呼ぶ。
        """
        prev_theta = self.theta.copy()
        prev_vel = self.theta_vel.copy()
        prev_acc = self.theta_acc.copy()

        a = np.asarray(action).reshape(-1)
        if a.shape[0] != 2:
            raise ValueError(f"action must have shape (2,), got {a.shape}")

        a_clipped = np.clip(a, np.radians(ACTION_MIN), np.radians(ACTION_MAX))
        action_same = bool(np.allclose(a_clipped[0], a_clipped[1], atol=self._same_action_tol))
        if action_same:
            self._same_action_counter += 1
        else:
            self._same_action_counter = 0

        # 関節角度更新（簡易: 角速度指令 * dt）
        self.theta = self.theta + a_clipped * self.dt
        self.theta = np.clip(self.theta, np.radians(THETA_MIN), np.radians(THETA_MAX))

        # 関節微分量更新（有限差分）
        self.update_joint_dynamics(prev_theta, prev_vel, prev_acc)

        # 手先の運動（有限差分）
        prev_hand_pos = self.hand_pos.copy()
        prev_hand_vel = self.hand_vel.copy()
        prev_hand_acc = self.hand_acc.copy()

        self.hand_pos = self.forward_kinematics(self.theta)
        self.hand_vel = (self.hand_pos - prev_hand_pos) / self.dt
        self.hand_acc = (self.hand_vel - prev_hand_vel) / self.dt
        self.hand_jerk = (self.hand_acc - prev_hand_acc) / self.dt

        hand_jerk_norm_sq = float(np.dot(self.hand_jerk, self.hand_jerk))
        self.jerk_sum += hand_jerk_norm_sq * self.dt

        self.steps += 1
        self.t += self.dt

        dist_to_goal = float(np.linalg.norm(self.hand_pos - self.goal_pos))

        # shaping 報酬
        reward_dist_step = self.shaping_dist_coeff * (self.prev_dist - dist_to_goal)
        reward_jerk_step = - self.shaping_jerk_coeff * hand_jerk_norm_sq * self.dt
        reward_time_step = - self.time_cost
        reward = reward_dist_step + reward_jerk_step + reward_time_step

        # 進捗ウィンドウ更新
        self.dist_window.append(dist_to_goal)
        if len(self.dist_window) > self.stall_window:
            self.dist_window.pop(0)

        terminated = False
        truncated = False

        terminal_jerk_penalty = 0.0
        terminal_vel_penalty = 0.0
        time_bonus = 0.0

        if self.reached_goal(dist_to_goal):
            # 時間ボーナスの実装（ガウス型：中心 T_target、幅 sigma_T）
            # note: sigma_T は self.sigma_T を使う（これが動的に更新される）
            time_bonus = self.R_goal * np.exp(- (self.t - self.T_target)**2 / (2 * max(1e-12, (self.sigma_T**2))))
            terminal_jerk_penalty = - self.terminal_jerk_coeff * (self.jerk_sum / max(1, self.steps))
            hand_vel_norm_sq = float(np.dot(self.hand_vel, self.hand_vel))
            terminal_vel_penalty = - self.terminal_vel_coeff * hand_vel_norm_sq

            reward += time_bonus + terminal_jerk_penalty + terminal_vel_penalty
            terminated = True

        elif len(self.dist_window) >= self.stall_window:
            prog = self.dist_window[0] - self.dist_window[-1]
            if prog < self.min_progress_per_window:
                truncated = True
                reward += -0.5

        if not terminated and self.steps >= self.max_steps:
            truncated = True
            reward += - self.terminal_jerk_coeff * (self.jerk_sum / max(1, self.steps))

        reward_components = {
            "reward_dist_step": float(reward_dist_step),
            "reward_jerk_step": float(reward_jerk_step),
            "reward_time_step": float(reward_time_step),
            "terminal_jerk_penalty": float(terminal_jerk_penalty),
            "terminal_vel_penalty": float(terminal_vel_penalty),
            "time_bonus": float(time_bonus),
            "jerk_sum": float(self.jerk_sum),
            "hand_jerk_norm_sq": hand_jerk_norm_sq
        }

        info = {
            "hand_pos": self.hand_pos,
            "hand_vel": self.hand_vel,
            "hand_acc": self.hand_acc,
            "hand_jerk": self.hand_jerk,
            "theta": self.theta.copy(),
            "theta_vel": self.theta_vel.copy(),
            "theta_acc": self.theta_acc.copy(),
            "theta_jerk": self.theta_jerk.copy(),
            "dist_to_goal": dist_to_goal,
            "t": float(self.t),
            "action": a_clipped.copy(),
            "action_same": action_same,
            "same_action_counter": int(self._same_action_counter),
            "reward_total": float(reward),
            **reward_components,
            
        }

        # --- エピソード終端なら success-window を更新し、必要なら sigma を更新 ---
        if terminated or truncated:
            success_flag = 1 if terminated else 0
            self._success_window.append(success_flag)

            sigma_updated = False
            # --- 修正箇所: ウィンドウが満杯かつ interval に到達したときだけ update を呼ぶ ---
            if len(self._success_window) == self._success_window_size:
                # カウンタをインクリメント（ウィンドウが満杯になってからカウント）
                self._sigma_update_counter += 1
                # interval に到達したタイミングでのみ更新判定を行う
                if (self._sigma_update_counter % self.sigma_update_interval) == 0:
                    sigma_updated = self.update_sigma_by_success_rate()
            # info に sigma 情報を付加して返す（ログしやすくする）
            info["sigma_T"] = float(self.sigma_T)
            info["sigma_update"] = dict(self._last_sigma_update)

        else:
            info["sigma_T"] = float(self.sigma_T)
            info["sigma_update"] = dict(self._last_sigma_update)

        self.prev_dist = dist_to_goal

        return self._get_obs(), float(reward), bool(terminated), bool(truncated), info

    # ---------- 関節微分量更新 ----------
    def update_joint_dynamics(self, prev_theta, prev_vel, prev_acc):
        """角速度・角加速度・角躍度を有限差分で更新する（シンプル実装）"""
        self.theta_vel = (self.theta - prev_theta) / self.dt
        self.theta_acc = (self.theta_vel - prev_vel) / self.dt
        self.theta_jerk = (self.theta_acc - prev_acc) / self.dt

    # ---------- 描画 ----------
    def render(self):
        """簡易描画: コンソールに手先や角度を出力"""
        print(
            f"t={self.t:.3f}s step={self.steps}, "
            f"theta1={np.degrees(self.theta[0]):.2f} deg, theta2={np.degrees(self.theta[1]):.2f} deg, "
            f"hand={self.hand_pos}, sigma_T={self.sigma_T:.6f}"
        )


## 環境が動くかの確認

In [ ]:
env = TwoJointReachingEnv()
check_env(env, warn=True)

## エピソードごとの報酬などの学習内部指標を定期的に保存するためのクラス

In [ ]:
class FullMonitorCallback(BaseCallback):
    """
    エピソード指標 + 学習内部指標（actor_loss, critic_loss, ent_coef, avg_q など）を
    CSV に定期的に保存するコールバック（SB3 用）。

    追加機能:
      - CSV の最終列に env.info に含まれる "sigma_T" を書き込む（存在すれば）
      - エピソード内で info に含まれるアクション分布の標準偏差をステップ毎に収集し、
        その**平均値**をエピソードごとに計算して CSV の最終列に "mean_action_std" として保存
    """

    def __init__(self,
                 log_dir: str,
                 save_interval = SAVE_INTERVAL,
                 running_window = RUNNNIG_WINDOW,
                 verbose: int = 0):
        super().__init__(verbose)
        self.log_dir = log_dir
        os.makedirs(self.log_dir, exist_ok=True)
        self.save_interval = int(save_interval)
        self.running_window = int(running_window)

        # CSVファイルのパス（固定）
        self.csv_path = os.path.join(self.log_dir, "episode_full_metrics.csv")

        # 基本ヘッダ（最後に sigma_T と mean_action_std を追加）
        header = [
            "episode",
            "episode_length",
            "total_reward",
            "sum_reward_dist_step",
            "sum_reward_jerk_step",
            "sum_reward_time_step",
            "sum_terminal_jerk_penalty",
            "sum_terminal_vel_penalty",
            "sum_time_bonus",
            "jerk_sum",
            "success",
            "actor_loss",
            "critic_loss",
            "ent_coef",
            "ent_coef_loss",
            f"running_mean_total_reward_{self.running_window}",
            f"running_success_rate_{self.running_window}",
            "episode_wall_time",
            # --- 追加項目（末尾へ） ---
            "mean_action_std",   # 1エピソード内の action-std の平均（ない場合 NaN）
            "sigma_T"            # info["sigma_T"] の値（ない場合 NaN）
        ]

        # ヘッダーを書き込む（ファイル上書き）
        with open(self.csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)

        # 内部バッファ（エピソード単位）
        self._step_rewards = []     # 各ステップ報酬
        self._step_infos = []       # 各ステップ info の shallow copy
        self._step_action_stds = [] # 各ステップで見つかった action-std の値（float）
        self.episode_count = 0
        self.episode_records = []   # running_window に使うサマリ
        self._ep_start_time = None
        self._train_start_time = None

        # logger key 候補（action_std を探すための key 候補群）
        self._action_std_keys = [
            "action_std", "action_sigma", "pi_std", "policy_std", "std",
            "stddev", "log_std", "scale", "action_dist_std"
        ]

        # その他 logger key 候補（元実装のまま）
        self._actor_loss_keys = ["train/actor_loss", "actor_loss", "train/actor_loss/mean"]
        self._critic_loss_keys = ["train/critic_loss", "critic_loss", "train/critic_loss/mean"]
        self._ent_coef_keys = ["train/ent_coef", "ent_coef", "train/entropy_coef"]
        self._ent_coef_loss_keys = ["train/ent_coef_loss", "ent_coef_loss", "ent_coef/grad"]
        self._avg_q_keys = ["train/average_q", "train/mean_q", "train/q_value", "average_q"]

    def _on_training_start(self) -> None:
        self._train_start_time = time.time()

    def _on_step(self) -> bool:
        # locals から infos / rewards を取得（VecEnv の場合はリストになる）
        infos = self.locals.get("infos")
        rewards = self.locals.get("rewards")

        # infos の取り出し（最初の env を対象）
        if isinstance(infos, (list, tuple, np.ndarray)):
            info = infos[0] if len(infos) > 0 else {}
        else:
            info = infos or {}

        # rewards の取り出し（最初の env を対象）
        if isinstance(rewards, (list, tuple, np.ndarray)):
            reward = float(rewards[0])
        else:
            reward = float(rewards or 0.0)

        # ステップデータをバッファへ追加
        self._step_rewards.append(reward)
        self._step_infos.append(dict(info))  # shallow copy

        # --- アクション std を info 内から探す（複数候補） ---
        std_val = None
        if info:
            for key in self._action_std_keys:
                if key in info:
                    val = info.get(key)
                    try:
                        arr = np.asarray(val)
                        # 配列なら平均絶対値（要素ごとの std のときの扱い）
                        if arr.size > 1:
                            std_val = float(np.mean(np.abs(arr)))
                        else:
                            std_val = float(arr.item())
                    except Exception:
                        try:
                            std_val = float(val)
                        except Exception:
                            std_val = None
                    break
            # もし info 内に "action_dist" のような辞書が入っているケースを探る（便利なら）
            if std_val is None:
                # common nested patterns
                nested = info.get("action_dist") or info.get("dist") or info.get("policy_dist")
                if isinstance(nested, dict):
                    for key in self._action_std_keys:
                        if key in nested:
                            try:
                                arr = np.asarray(nested[key])
                                std_val = float(np.mean(np.abs(arr))) if arr.size > 1 else float(arr.item())
                                break
                            except Exception:
                                pass

        # ステップごとの action-std を記録（存在しないステップはスキップ）
        if std_val is not None:
            self._step_action_stds.append(std_val)

        # dones を調べてエピソード終了判定（最初の env を対象）
        dones = self.locals.get("dones")
        if isinstance(dones, (list, tuple, np.ndarray)):
            done_flag = bool(dones[0])
        else:
            done_flag = bool(dones)

        # エピソード開始時のタイマー設定
        if self._ep_start_time is None:
            self._ep_start_time = time.time()

        # エピソード終了処理
        if done_flag:
            self.episode_count += 1
            ep_len = len(self._step_rewards)
            total_reward = float(np.sum(self._step_rewards)) if self._step_rewards else 0.0

            # helper to sum info key across steps
            def sum_info_key(key: str) -> float:
                return float(sum([it.get(key, 0.0) for it in self._step_infos]))

            sum_reward_dist_step = sum_info_key("reward_dist_step")
            sum_reward_jerk_step = sum_info_key("reward_jerk_step")
            sum_reward_time_step = sum_info_key("reward_time_step")
            sum_terminal_jerk_penalty = sum_info_key("terminal_jerk_penalty")
            sum_terminal_vel_penalty = sum_info_key("terminal_vel_penalty")
            sum_time_bonus = sum_info_key("time_bonus")

            # jerk_sum の取得（info に直接あればそれを使い、なければ近似）
            jerk_sum = None
            if self._step_infos:
                if "jerk_sum" in self._step_infos[-1]:
                    jerk_sum = float(self._step_infos[-1].get("jerk_sum", 0.0))
                else:
                    # try to approximate from theta_jerk using env.dt if available
                    dt = None
                    try:
                        base_env = getattr(self.training_env, "envs", [None])[0]
                        dt = getattr(base_env, "dt", None)
                    except Exception:
                        dt = None
                    if dt is None:
                        dt = 0.01
                    jerk_acc = 0.0
                    for it in self._step_infos:
                        tj = float(it.get("theta_jerk", 0.0))
                        jerk_acc += (tj ** 2) * float(dt)
                    jerk_sum = jerk_acc

            # success 判定
            success = bool(self._step_infos[-1].get("success", False)) if self._step_infos else False
            if not success:
                last_dist = float(self._step_infos[-1].get("dist_to_goal", np.inf)) if self._step_infos else np.inf
                try:
                    base_env = getattr(self.training_env, "envs", [None])[0]
                    goal_thresh = getattr(base_env, "goal_threshold", None)
                    if goal_thresh is None:
                        goal_thresh = getattr(base_env, "THRESHOLD", None)
                    if goal_thresh is not None:
                        success = last_dist <= float(goal_thresh)
                except Exception:
                    pass

            # エピソード経過時間（wall time）
            ep_wall_time = time.time() - (self._ep_start_time or time.time())
            self._ep_start_time = None

            # running window summary
            self.episode_records.append({
                "episode": self.episode_count,
                "episode_length": ep_len,
                "total_reward": total_reward,
                "success": int(success),
                "jerk_sum": jerk_sum
            })
            last_N = self.episode_records[-self.running_window:]
            running_mean_total_reward = float(np.mean([r["total_reward"] for r in last_N])) if last_N else 0.0
            running_success_rate = float(np.mean([r["success"] for r in last_N])) if last_N else 0.0

            # 学習内部指標の取得（logger から）
            actor_loss = np.nan; critic_loss = np.nan; ent_coef = np.nan; ent_coef_loss = np.nan; avg_q = np.nan
            name_to_value = {}
            try:
                if hasattr(self.model, "logger") and hasattr(self.model.logger, "name_to_value"):
                    name_to_value = getattr(self.model.logger, "name_to_value", {}) or {}
                elif hasattr(self.logger, "name_to_value"):
                    name_to_value = getattr(self.logger, "name_to_value", {}) or {}
            except Exception:
                name_to_value = {}

            def pick_first_key(candidates: List[str]) -> Optional[float]:
                for k in candidates:
                    if k in name_to_value:
                        try:
                            return float(name_to_value.get(k))
                        except Exception:
                            pass
                return None

            val = pick_first_key(self._actor_loss_keys)
            if val is not None:
                actor_loss = val
            val = pick_first_key(self._critic_loss_keys)
            if val is not None:
                critic_loss = val
            val = pick_first_key(self._ent_coef_keys)
            if val is not None:
                ent_coef = val
            val = pick_first_key(self._ent_coef_loss_keys)
            if val is not None:
                ent_coef_loss = val
            val = pick_first_key(self._avg_q_keys)
            if val is not None:
                avg_q = val

            # --- 追加: エピソード内の action-std の平均を計算 ---
            if len(self._step_action_stds) > 0:
                mean_action_std = float(np.nanmean(self._step_action_stds))
            else:
                mean_action_std = float("nan")

            # --- 追加: sigma_T を取得 (info の最後のステップに入っている想定) ---
            sigma_T_val = float(self._step_infos[-1].get("sigma_T", float("nan"))) if self._step_infos else float("nan")

            # CSV 行作成（末尾に mean_action_std と sigma_T を追加）
            row = [
                self.episode_count,
                ep_len,
                total_reward,
                sum_reward_dist_step,
                sum_reward_jerk_step,
                sum_reward_time_step,
                sum_terminal_jerk_penalty,
                sum_terminal_vel_penalty,
                sum_time_bonus,
                jerk_sum,
                int(success),
                actor_loss,
                critic_loss,
                ent_coef,
                ent_coef_loss,
                running_mean_total_reward,
                running_success_rate,
                ep_wall_time,
                # 末尾追加
                mean_action_std,
                sigma_T_val
            ]

            # save_interval ごとにファイルへ追記
            if (self.episode_count % self.save_interval) == 0:
                with open(self.csv_path, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(row)
                if self.verbose:
                    print(f"[FullMonitor] Saved episode {self.episode_count} to {self.csv_path}")

            # SB3 logger にも記録
            try:
                self.logger.record("episode/total_reward", total_reward)
                self.logger.record("episode/jerk_sum", jerk_sum)
                self.logger.record("episode/length", ep_len)
                self.logger.record("episode/success", int(success))
                self.logger.record("episode/running_mean_reward", running_mean_total_reward)
                self.logger.record("episode/running_success_rate", running_success_rate)
                # record action-std and sigma
                self.logger.record("episode/mean_action_std", mean_action_std)
                self.logger.record("episode/sigma_T", sigma_T_val)
                if not np.isnan(actor_loss):
                    self.logger.record("train/actor_loss", float(actor_loss))
                if not np.isnan(critic_loss):
                    self.logger.record("train/critic_loss", float(critic_loss))
                if not np.isnan(ent_coef):
                    self.logger.record("train/ent_coef", float(ent_coef))
                if not np.isnan(avg_q):
                    self.logger.record("train/avg_q", float(avg_q))
            except Exception:
                pass

            # エピソードバッファをクリア
            self._step_rewards.clear()
            self._step_infos.clear()
            self._step_action_stds.clear()

        return True


## 学習コード
### SAC モデルと EvalCallback の設定

ここでは、SAC (Soft Actor-Critic) を用いて `OneJointReachingEnv` を学習します。  
重要なオプションとその意味は以下の通りです：
- **total_timesteps**: 学習に使う総ステップ数（エピソードではなく「1ステップ」の数）
- **eval_freq**: 何ステップごとに評価を行うか（例：2000 → 2000ステップごと）
- **best_model_save_path**: 評価が一番良かったモデルを保存するフォルダ
- **deterministic**: 評価時に行動を決定的にするか（学習したポリシーの実力確認用）
- **render**: 評価時に描画するか（Trueにすると遅くなるが動きが見える）

参考: [Stable-Baselines3 EvalCallback](https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html#evalcallback)

### 学習ログの見方
```plaintext
| episode/                |          
|    jerk_sum             | １エピソードの躍度の総和 
|    length               | １エピソードのステップ数       
|    mean_action_std      | actor-networkの出力値（開発中）     
|    running_mean_reward  | 報酬の総和の平均
|    running_success_rate | 到達成功率（開発中）
|    sigma_T              | 躍度における報酬付与の変化の度合い
|    success              | 到達成功
|    total_reward         | 報酬の総和
| eval/                   |          
|    mean_ep_length       | エピソードの平均ステップ数
|    mean_reward          | 報酬の総和の平均（ステップ数を変えた場合）
| time/                   |          
|    total_timesteps      | 学習で実行した環境ステップの総数
--------------------------------------

In [ ]:
# ✅ 実行ごとに一意なディレクトリを作成（日時で名前をつける）
run_id = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
log_dir = f"./logs/{run_id}"
os.makedirs(log_dir, exist_ok=True)

# ✅ パラメータを保存しておく
training_params = {
    "DT":DT,
    "L" : L,           # アームの長さ (m)
    "THETA_INIT" :THETA_INIT,        # 初期角度 [deg]""
    "GOAL_POS" : GOAL_POS,              # ゴール位置 (x, y)
    "STEPS_MAX" : STEPS_MAX,     # 1エピソードあたりの最大ステップ数
    "THRESHOLD" : THRESHOLD,    # ゴール判定の許容距離 (m)  
    "REWARD_P_V" : REWARD_P_V,        # 速度ペナルティの重み
    "REWARD_J" : REWARD_J,   # 躍度ペナルティの重み
    "REWARD_JE_LIM" : REWARD_JE_LIM,     # 躍度ペナルティの上限
    "T_TARGET" : T_TARGET,
    "SIGMA_T" : SIGMA_T,
    "R_GOAL" : R_GOAL,
    "TIME_COST" : TIME_COST,
    "SHAPING_DIST_COEFF" : SHAPING_DIST_COEFF,     # 距離改善に対するステップ報酬倍率
    "SHAPING_JERK_COEFF" : SHAPING_JERK_COEFF,   # 瞬時躍度ペナルティ係数（小さく）
    "total_timesteps": TOTAL_TIMESTEPS,
    "learning_rate": LEARNING_RATE,
    "buffer_size": BUFFER_SIZE,
    "batch_size": BATCH_SIZE,
    "eval_freq": EVAL_FREQ,
    "hid_lay": HID_LAY,
    "TAU": TAU,
}


with open(os.path.join(log_dir, "params.json"), "w") as f:
    json.dump(training_params, f, indent=4)

# ✅ 環境を作成
env = TwoJointReachingEnv()
eval_env = TwoJointReachingEnv()


# ✅ EvalCallback のベストモデル保存先を日時付きに変更
eval_callback = EvalCallback(
    eval_env,                               # 評価用環境
    best_model_save_path=log_dir,           # ベストモデルを保存するディレクトリ
    log_path=log_dir,                       # 評価結果をログ保存するディレクトリ
    eval_freq=training_params["eval_freq"], # 何ステップごとに評価するか
    deterministic=True,                     # 評価時は行動を確率的にせず決定論的にする
    render=False                            # 評価中に画面描画するか（通常False）
)

# ✅ SACモデル作成
policy_kwargs = dict(net_arch=[HID_LAY, HID_LAY])


# model = SAC(
#     "MlpPolicy",                                     # ポリシーネットワークの種類（多層パーセプトロン）
#     env,                                             # 学習環境
#     verbose=1,                                       # ログ出力レベル
#     learning_rate=training_params["learning_rate"],  # 学習率（必要に応じて変更可
#     buffer_size=training_params["buffer_size"],      # リプレイバッファサイズ
#     batch_size=training_params["batch_size"],        # バッチサイズ
#     tau=training_params["TAU"],
#     policy_kwargs=policy_kwargs
# )

model = PPO(
    "MlpPolicy",                                     # ポリシーネットワークの種類（多層パーセプトロン）
    env,                                             # 学習環境
    verbose=1,                                       # ログ出力レベル
    learning_rate=training_params["learning_rate"],  # 学習率（必要に応じて変更可
    batch_size=training_params["batch_size"],        # バッチサイズ
    policy_kwargs=policy_kwargs                      #方策ネットワークの構成を渡す
)


# ✅ 学習実行（EvalCallbackと報酬記録を組み合わせる）
callback=[eval_callback, FullMonitorCallback(log_dir,verbose=1)]

model.learn(
    total_timesteps=training_params["total_timesteps"], # 総ステップ数（ここを変えれば学習量を調整できる）
    callback=callback                                   # 評価コールバック
)

# ===== 学習終了時に final model を保存する=====
final_model_path = os.path.join(log_dir, "final_model.zip")
model.save(final_model_path)
print(f"✅ Final model saved: {final_model_path}")



## 学習終了時点のモデルの活用

In [ ]:
# -------------------------
# 設定: 最新ログフォルダを取得（保存先）
# -------------------------
logs_dir = "./logs"
log_folders = [os.path.join(logs_dir, d) for d in os.listdir(logs_dir) if os.path.isdir(os.path.join(logs_dir, d))]
latest_log_folder = max(log_folders, key=os.path.getmtime)
print(f"[INFO] 最新ログフォルダを使用: {latest_log_folder}")

# -------------------------
# ログ用リスト初期化（2関節 + 手先ノルム）
# -------------------------
time_log = []
hand_x, hand_y = [], []

# 関節ごとのログ
theta1_log, theta2_log = [], []
theta1_vel_log, theta2_vel_log = [], []
theta1_acc_log, theta2_acc_log = [], []
theta1_jerk_log, theta2_jerk_log = [], []

# action とチェック用
action1_log, action2_log = [], []
action_same_log = []

# 手先ノルム系ログ（追加）
hand_speed_log = []       # ||v||
hand_acc_norm_log = []    # ||a||
hand_jerk_norm_log = []   # ||jerk||

# ----- 初期観測を取得して初期状態をログに追加 -----
obs, _ = env.reset()

try:
    init_theta = np.asarray(env.theta).reshape(-1)
    init_theta_vel = np.asarray(env.theta_vel).reshape(-1)
    init_theta_acc = np.asarray(env.theta_acc).reshape(-1)
    init_theta_jerk = np.asarray(env.theta_jerk).reshape(-1)
    init_hand_pos = np.asarray(getattr(env, "hand_pos", env.forward_kinematics(init_theta)))
    init_hand_vel = np.asarray(getattr(env, "hand_vel", np.zeros(2)))
    init_hand_acc = np.asarray(getattr(env, "hand_acc", np.zeros(2)))
    init_hand_jerk = np.asarray(getattr(env, "hand_jerk", np.zeros(2)))
except Exception:
    # フォールバック
    init_theta = np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])
    init_theta_vel = np.zeros(2)
    init_theta_acc = np.zeros(2)
    init_theta_jerk = np.zeros(2)
    init_hand_pos = np.asarray(getattr(env, "hand_pos", np.array([0.0, 0.0])))
    init_hand_vel = np.zeros(2)
    init_hand_acc = np.zeros(2)
    init_hand_jerk = np.zeros(2)

# 時刻ゼロを設定して初期値を格納（deg に変換して保存）
t = 0.0
time_log.append(t)
hand_x.append(float(init_hand_pos[0]))
hand_y.append(float(init_hand_pos[1]))

theta1_log.append(np.degrees(init_theta[0]))
theta2_log.append(np.degrees(init_theta[1]))
theta1_vel_log.append(np.degrees(init_theta_vel[0]))
theta2_vel_log.append(np.degrees(init_theta_vel[1]))
theta1_acc_log.append(np.degrees(init_theta_acc[0]))
theta2_acc_log.append(np.degrees(init_theta_acc[1]))
theta1_jerk_log.append(np.degrees(init_theta_jerk[0]))
theta2_jerk_log.append(np.degrees(init_theta_jerk[1]))

# 初期の手先ノルム
hand_speed_log.append(float(np.linalg.norm(init_hand_vel)))
hand_acc_norm_log.append(float(np.linalg.norm(init_hand_acc)))
hand_jerk_norm_log.append(float(np.linalg.norm(init_hand_jerk)))

action1_log.append(np.nan)
action2_log.append(np.nan)
action_same_log.append(False)

# ===== 実行ループ =====
max_steps_to_run = 200
for _ in range(max_steps_to_run):
    # 学習済みモデルから行動（2次元）を取得
    action, _ = model.predict(obs, deterministic=True)
    action = np.asarray(action).reshape(-1)
    if action.shape[0] != 2:
        raise RuntimeError(f"model.predict returned action with wrong shape {action.shape}. expected (2,)")

    obs, reward, done, truncated, info = env.step(action)

    # 時刻進行
    t += getattr(env, "dt", 0.01)
    time_log.append(t)

    # info を優先して手先・関節情報を取得、なければ env 属性を参照
    if info is not None:
        hand_pos = np.asarray(info.get("hand_pos", getattr(env, "hand_pos", np.array([0.0, 0.0]))))
        hand_vel = np.asarray(info.get("hand_vel", getattr(env, "hand_vel", np.zeros(2))))
        hand_acc = np.asarray(info.get("hand_acc", getattr(env, "hand_acc", np.zeros(2))))
        hand_jerk = np.asarray(info.get("hand_jerk", getattr(env, "hand_jerk", np.zeros(2))))

        theta = np.asarray(info.get("theta", getattr(env, "theta", np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)]))))
        theta_vel = np.asarray(info.get("theta_vel", getattr(env, "theta_vel", np.zeros(2))))
        theta_acc = np.asarray(info.get("theta_acc", getattr(env, "theta_acc", np.zeros(2))))
        theta_jerk = np.asarray(info.get("theta_jerk", getattr(env, "theta_jerk", np.zeros(2))))

        action_info = np.asarray(info.get("action", action))
        action_same = bool(info.get("action_same", np.allclose(action_info[0], action_info[1], atol=1e-8)))
    else:
        # フォールバック
        hand_pos = np.asarray(getattr(env, "hand_pos", np.array([0.0,0.0])))
        hand_vel = np.asarray(getattr(env, "hand_vel", np.zeros(2)))
        hand_acc = np.asarray(getattr(env, "hand_acc", np.zeros(2)))
        hand_jerk = np.asarray(getattr(env, "hand_jerk", np.zeros(2)))

        theta = np.asarray(getattr(env, "theta", np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])))
        theta_vel = np.asarray(getattr(env, "theta_vel", np.zeros(2)))
        theta_acc = np.asarray(getattr(env, "theta_acc", np.zeros(2)))
        theta_jerk = np.asarray(getattr(env, "theta_jerk", np.zeros(2)))

        action_info = np.asarray(action)
        action_same = bool(np.allclose(action_info[0], action_info[1], atol=1e-8))

    # 手先位置
    hand_x.append(float(hand_pos[0]))
    hand_y.append(float(hand_pos[1]))

    # 関節（deg）に変換してログ
    theta1_log.append(np.degrees(theta[0])); theta2_log.append(np.degrees(theta[1]))
    theta1_vel_log.append(np.degrees(theta_vel[0])); theta2_vel_log.append(np.degrees(theta_vel[1]))
    theta1_acc_log.append(np.degrees(theta_acc[0])); theta2_acc_log.append(np.degrees(theta_acc[1]))
    theta1_jerk_log.append(np.degrees(theta_jerk[0])); theta2_jerk_log.append(np.degrees(theta_jerk[1]))

    # action を deg/s 表示で保存
    action_deg = np.degrees(action_info)
    action1_log.append(float(action_deg[0])); action2_log.append(float(action_deg[1]))
    action_same_log.append(bool(action_same))

    # --- 手先ノルムを計算してログ ---
    hand_speed = float(np.linalg.norm(hand_vel))     # ||v||
    hand_acc_norm = float(np.linalg.norm(hand_acc))  # ||a||
    hand_jerk_norm = float(np.linalg.norm(hand_jerk))# ||jerk||

    hand_speed_log.append(hand_speed)
    hand_acc_norm_log.append(hand_acc_norm)
    hand_jerk_norm_log.append(hand_jerk_norm)

    if bool(done) or bool(truncated):
        break

# ===== DataFrame に変換（手先ノルム列を追加） =====
df = pd.DataFrame({
    "Time": time_log,
    "HandX": hand_x,
    "HandY": hand_y,
    "HandSpeed (m/s)": hand_speed_log,
    "HandAcc (m/s^2)": hand_acc_norm_log,
    "HandJerk (m/s^3)": hand_jerk_norm_log,
    "Theta1 (deg)": theta1_log,
    "Theta2 (deg)": theta2_log,
    "Theta1_vel (deg/s)": theta1_vel_log,
    "Theta2_vel (deg/s)": theta2_vel_log,
    "Theta1_acc (deg/s^2)": theta1_acc_log,
    "Theta2_acc (deg/s^2)": theta2_acc_log,
    "Theta1_jer (deg/s^3)": theta1_jerk_log,
    "Theta2_jer (deg/s^3)": theta2_jerk_log,
    "Action1 (deg/s)": action1_log,
    "Action2 (deg/s)": action2_log,
    "Action_same": action_same_log
})

# ===== 保存 =====
output_folder = latest_log_folder
os.makedirs(output_folder, exist_ok=True)
base_filename = os.path.join(output_folder, "end_result_2joint_with_handnorms")
csv_path = f"{base_filename}.csv"
df.to_csv(csv_path, index=False)
print(f"[INFO] CSV 保存: {csv_path}")

# ===== プロット: 手先軌道 =====
# ===== プロット: 手先軌道 + アーム姿勢を重ねる =====
fig1, ax1 = plt.subplots(figsize=(6,6))

# shoulder を原点に固定
shoulder = np.array([0.0, 0.0])

# 手先軌道（青ライン）
ax1.plot(df["HandX"], df["HandY"], linestyle="-", linewidth=1.5, label="end-effector trajectory")

# start / end markers
ax1.scatter(df["HandX"].iloc[0], df["HandY"].iloc[0], color="red", label="start", zorder=5)
ax1.scatter(df["HandX"].iloc[-1], df["HandY"].iloc[-1], color="green", label="end", zorder=5)

# リンク長（env から取得できなければ L を半分ずつ使う）
try:
    l1 = float(getattr(env, "l1", (L*0.5)))
    l2 = float(getattr(env, "l2", (L*0.5)))
except Exception:
    l1 = L * 0.5
    l2 = L * 0.5

# 関節位置を計算（theta は deg で保存されている想定）
theta1_rad = np.radians(df["Theta1 (deg)"].values)
theta2_rad = np.radians(df["Theta2 (deg)"].values)

# 第1関節（肘ではなくリンク1の末端）位置
x1 = l1 * np.cos(theta1_rad)
y1 = l1 * np.sin(theta1_rad)
# 手先（リンク2末端）
x2 = x1 + l2 * np.cos(theta1_rad + theta2_rad)
y2 = y1 + l2 * np.sin(theta1_rad + theta2_rad)

# 確認：手先ログと一致するはず
# (もし mismatch があれば警告を出す)
try:
    mismatch = np.nanmax(np.abs(np.column_stack([df["HandX"].values, df["HandY"].values]) - np.column_stack([x2, y2])))
    if mismatch > 1e-6:
        # print 小さめの警告（実行中に表示）
        print(f"[WARN] forward kinematics and logged hand pos mismatch (max diff={mismatch:.6e})")
except Exception:
    pass

# アーム姿勢を複数タイムスタンプで描く
N_POSES = min(15, len(df))  # 最大描画姿勢数
indices = np.linspace(0, len(df)-1, N_POSES).astype(int)

# 薄いグレーで時系列に沿ってアームをプロット（透過）
for idx in indices:
    xs = [0.0, x1[idx], x2[idx]]
    ys = [0.0, y1[idx], y2[idx]]
    ax1.plot(xs, ys, linewidth=1, color=(0.5,0.5,0.5,0.25), solid_capstyle='round')  # 太めの線
    ax1.scatter([xs[1], xs[2]], [ys[1], ys[2]], s=40, edgecolors='k', facecolors=(0.9,0.9,0.9,0.6), zorder=4)

# 強調: start, mid, end の姿勢（色とマーカーで分かりやすく）
# start
i0 = 0
xs0 = [0.0, x1[i0], x2[i0]]
ys0 = [0.0, y1[i0], y2[i0]]
ax1.plot(xs0, ys0, linewidth=2, color='tab:red', solid_capstyle='round', zorder=6)
ax1.scatter([xs0[1], xs0[2]], [ys0[1], ys0[2]], s=80, color='tab:red', edgecolors='k', zorder=7)
ax1.scatter(0.0, 0.0, s=100, color='black', zorder=8)  # shoulder


# end
ie = len(df)-1
xse = [0.0, x1[ie], x2[ie]]
yse = [0.0, y1[ie], y2[ie]]
ax1.plot(xse, yse, linewidth=2, color='tab:green', solid_capstyle='round', zorder=6)
ax1.scatter([xse[1], xse[2]], [yse[1], yse[2]], s=100, color='tab:green', edgecolors='k', zorder=7)

# # 関節角度の数値も近傍に表示（start と end のみ、必要なら増やせます）
# ang_off = 0.03 * (l1 + l2)  # 表示オフセット
# ax1.text(xs0[1]+ang_off, ys0[1]+ang_off, f"{np.degrees(theta1_rad[i0]):.1f}°", color='tab:red', fontsize=9)
# ax1.text(xs0[2]+ang_off, ys0[2]+ang_off, f"{np.degrees(theta2_rad[i0]):.1f}°", color='tab:red', fontsize=9)
# ax1.text(xse[1]+ang_off, yse[1]+ang_off, f"{np.degrees(theta1_rad[ie]):.1f}°", color='tab:green', fontsize=9)
# ax1.text(xse[2]+ang_off, yse[2]+ang_off, f"{np.degrees(theta2_rad[ie]):.1f}°", color='tab:green', fontsize=9)

# 装飾
ax1.set_xlabel("X [m]"); ax1.set_ylabel("Y [m]")
# ax1.set_title("End-effector Trajectory with Arm Poses (shoulder at (0,0))")
ax1.grid(True)
ax1.set_aspect('equal', adjustable='box')

# 自動スケール（アーム長に沿ったマージンを付ける）
reach = l1 + l2
ax1.set_xlim(-reach-0.1*reach, reach+0.1*reach)
ax1.set_ylim(-0.1*reach, reach+0.6*reach)  # y 上方向に少し余裕

plt.tight_layout()
plt.savefig(f"{base_filename}_trajectory_with_arms.png", dpi=300)
plt.show()

# ===== プロット: 関節動的変数（Joint1 / Joint2 両表示） =====
fig2, axes = plt.subplots(4,1, figsize=(8,12), sharex=True)
axes[0].plot(df["Time"], df["Theta1 (deg)"], label="Theta1"); axes[0].plot(df["Time"], df["Theta2 (deg)"], linestyle="--", label="Theta2")
axes[0].set_ylabel("Theta (deg)"); axes[0].legend(); axes[0].grid()
axes[0].set_ylim(0, 180)

axes[1].plot(df["Time"], df["Theta1_vel (deg/s)"], label="Vel1"); axes[1].plot(df["Time"], df["Theta2_vel (deg/s)"], linestyle="--", label="Vel2")
axes[1].set_ylabel("Velocity (deg/s)"); axes[1].legend(); axes[1].grid()

axes[2].plot(df["Time"], df["Theta1_acc (deg/s^2)"], label="Acc1"); axes[2].plot(df["Time"], df["Theta2_acc (deg/s^2)"], linestyle="--", label="Acc2")
axes[2].set_ylabel("Acceleration (deg/s^2)"); axes[2].legend(); axes[2].grid()

axes[3].plot(df["Time"], df["Theta1_jer (deg/s^3)"], label="Jerk1"); axes[3].plot(df["Time"], df["Theta2_jer (deg/s^3)"], linestyle="--", label="Jerk2")
axes[3].set_ylabel("Jerk (deg/s^3)"); axes[3].set_xlabel("Time (s)"); axes[3].legend(); axes[3].grid()

plt.tight_layout()
plt.savefig(f"{base_filename}_joint_dynamics.png", dpi=300)
plt.show()

# ===== プロット: 手先ノルム（速度・加速度・躍度の大きさ） =====
fig3, ax3 = plt.subplots(3,1, figsize=(8,9), sharex=True)
ax3[0].plot(df["Time"], df["HandSpeed (m/s)"], label="Hand Speed ||v||")
ax3[0].set_ylabel("Speed (m/s)"); ax3[0].legend(); ax3[0].grid()

ax3[1].plot(df["Time"], df["HandAcc (m/s^2)"], label="Hand Acc ||a||")
ax3[1].set_ylabel("Acc (m/s^2)"); ax3[1].legend(); ax3[1].grid()

ax3[2].plot(df["Time"], df["HandJerk (m/s^3)"], label="Hand Jerk ||jerk||")
ax3[2].set_ylabel("Jerk (m/s^3)"); ax3[2].set_xlabel("Time (s)"); ax3[2].legend(); ax3[2].grid()

plt.tight_layout()
plt.savefig(f"{base_filename}_hand_norms.png", dpi=300)
plt.show()

# ===== 補助プロット: action の推移と action_same の確認（任意） =====
fig4, ax4 = plt.subplots(2,1, figsize=(8,6), sharex=True)
ax4[0].plot(df["Time"], df["Action1 (deg/s)"], label="Action1"); ax4[0].plot(df["Time"], df["Action2 (deg/s)"], linestyle="--", label="Action2")
ax4[0].set_ylabel("Action (deg/s)"); ax4[0].legend(); ax4[0].grid()
ax4[1].plot(df["Time"], df["Action_same"].astype(int), label="Action_same (int)")
ax4[1].set_ylabel("Action_same"); ax4[1].set_xlabel("Time (s)"); ax4[1].grid()
plt.tight_layout()
plt.savefig(f"{base_filename}_actions.png", dpi=300)
plt.show()


## ベストモデルの活用

In [ ]:
# -------------------------
# 1) 最新のログフォルダを見つけ、best_model.zip を探す
# -------------------------
log_base = "./logs"
log_dirs = [d for d in glob.glob(os.path.join(log_base, "*")) if os.path.isdir(d)]
if not log_dirs:
    raise FileNotFoundError("logs フォルダにサブディレクトリがありません。学習結果を保存してください。")

latest_log_dir = max(log_dirs, key=os.path.getmtime)
# best_model.zip がある想定。なければ最も新しい .zip を探すフォールバックを行う
best_model_path = os.path.join(latest_log_dir, "best_model.zip")
if not os.path.exists(best_model_path):
    # フォールバック: フォルダ内の .zip を探して一番新しいものを選ぶ
    zips = sorted(glob.glob(os.path.join(latest_log_dir, "*.zip")), key=os.path.getmtime)
    if not zips:
        raise FileNotFoundError(f"{best_model_path} が存在せず、{latest_log_dir} に .zip ファイルも見つかりません。")
    best_model_path = zips[-1]

print(f"✅ 最新のモデルファイルを使用: {best_model_path}")
best_model_path = "logs/2025_10_21_23_59_10/best_model.zip"

# -------------------------
# 便利関数: ラップを外してベース環境を得る
# -------------------------
def unwrap_env(e):
    """
    ラッパー (VecEnv, Monitor, DummyVecEnv など) があれば中身の base 環境を返す。
    再帰的に unwrap するので、どんなラップでもだいたい対応可能。
    """
    try:
        # VecEnv の場合: .envs がリスト
        if hasattr(e, "envs") and isinstance(getattr(e, "envs"), (list, tuple)) and len(e.envs) > 0:
            return unwrap_env(e.envs[0])
    except Exception:
        pass
    try:
        # Monitor / TimeLimit 等は .env を持つ場合がある
        if hasattr(e, "env") and e.env is not None:
            return unwrap_env(e.env)
    except Exception:
        pass
    return e

# -------------------------
# 2) best_model をロード（どのアルゴリズムか分からない場合を想定）
# -------------------------
loaded_model = None
# 試行順: PPO, SAC, TD3, DDPG, A2C
for alg in (PPO, SAC, TD3, DDPG, A2C):
    try:
        loaded_model = alg.load(best_model_path)
        print(f"✅ モデルを {alg.__name__} 形式でロードしました。")
        break
    except Exception as e:
        # 読み込み失敗は無視して次のアルゴリズムを試す
        # print(f"load with {alg.__name__} failed: {e}")
        pass

if loaded_model is None:
    raise RuntimeError("モデルのロードに失敗しました。対応する SB3 アルゴリズムを確認してください。")

best_model = loaded_model  # 名前を統一

# -------------------------
# 3) env のリセットと初期状態取得（env はユーザーのノートブックで定義済みの前提）
# -------------------------
# env が未定義だと失敗するので事前に env = TwoJointReachingEnv(...) を作成しておいてください
try:
    obs_reset = env.reset()
    # Gymnasium なら (obs, info), 古い Gym なら obs のみ。柔軟にハンドリング。
    if isinstance(obs_reset, tuple) and len(obs_reset) >= 1:
        obs = obs_reset[0]
    else:
        obs = obs_reset
except Exception as e:
    raise RuntimeError("env.reset() に失敗しました。env が正しく定義・初期化されているか確認してください.") from e

# unwrap して base_env を取る（内部状態読み出し用）
base_env = unwrap_env(env)

# -------------------------
# 4) ログ用リストを初期化（2関節対応）
# -------------------------
time_log = []
hand_x, hand_y = [], []

# 関節ごとのログ（joint1 / joint2）
theta1_log, theta2_log = [], []
theta1_vel_log, theta2_vel_log = [], []
theta1_acc_log, theta2_acc_log = [], []
theta1_jerk_log, theta2_jerk_log = [], []

# action ログ（deg/s 表示）
action1_log, action2_log = [], []
action_same_log = []

# 手先ノルムログ（速度, 加速度, 躍度）
hand_speed_log = []
hand_acc_norm_log = []
hand_jerk_norm_log = []

# -------------------------
# 5) 初期状態を取得してログに追加（時刻=0）
#    base_env が持つ属性を優先。なければ obs から推定（ただし obs 設計に依存するので注意）
# -------------------------
t = 0.0
time_log.append(t)

# try で base_env の配列属性を取り出す（theta は配列 [th1, th2] を期待）
try:
    base = base_env  # alias
    # theta 等は配列で返ることを期待（TwoJointReachingEnv の実装次第）
    init_theta = np.asarray(getattr(base, "theta", np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)]))).reshape(-1)[:2]
    init_theta_vel = np.asarray(getattr(base, "theta_vel", np.zeros(2))).reshape(-1)[:2]
    init_theta_acc = np.asarray(getattr(base, "theta_acc", np.zeros(2))).reshape(-1)[:2]
    init_theta_jerk = np.asarray(getattr(base, "theta_jerk", np.zeros(2))).reshape(-1)[:2]

    # hand_pos があれば優先して使う。無ければ順運動学か L を使って推定
    if hasattr(base, "hand_pos"):
        init_hand = np.asarray(getattr(base, "hand_pos")).astype(float)
    else:
        # try forward_kinematics if exists
        if hasattr(base, "forward_kinematics"):
            init_hand = base.forward_kinematics(init_theta)
        else:
            # fallback: 単純に L を手先長として theta[0] を使う (古い 1-joint 想定)
            L_val = float(getattr(base, "l", getattr(base, "l1", 1.0) + getattr(base, "l2", 0.0)))
            init_hand = L_val * np.array([np.cos(init_theta[0]), np.sin(init_theta[0])])
except Exception:
    # 最終フォールバック（安全）
    init_theta = np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])
    init_theta_vel = np.zeros(2)
    init_theta_acc = np.zeros(2)
    init_theta_jerk = np.zeros(2)
    try:
        init_hand = np.asarray(getattr(base_env, "hand_pos", np.array([0.0, 0.0])))
    except Exception:
        init_hand = np.array([0.0, 0.0])

# 初期値をログに追加（human-friendly に deg / SI 単位で）
hand_x.append(float(init_hand[0])); hand_y.append(float(init_hand[1]))
theta1_log.append(np.degrees(init_theta[0])); theta2_log.append(np.degrees(init_theta[1]))
theta1_vel_log.append(np.degrees(init_theta_vel[0])); theta2_vel_log.append(np.degrees(init_theta_vel[1]))
theta1_acc_log.append(np.degrees(init_theta_acc[0])); theta2_acc_log.append(np.degrees(init_theta_acc[1]))
theta1_jerk_log.append(np.degrees(init_theta_jerk[0])); theta2_jerk_log.append(np.degrees(init_theta_jerk[1]))

# 手先ノルム初期値（速度, acc, jerk）
init_hand_vel = np.asarray(getattr(base_env, "hand_vel", np.zeros(2))).reshape(-1)[:2]
init_hand_acc = np.asarray(getattr(base_env, "hand_acc", np.zeros(2))).reshape(-1)[:2]
init_hand_jerk = np.asarray(getattr(base_env, "hand_jerk", np.zeros(2))).reshape(-1)[:2]

hand_speed_log.append(float(np.linalg.norm(init_hand_vel)))
hand_acc_norm_log.append(float(np.linalg.norm(init_hand_acc)))
hand_jerk_norm_log.append(float(np.linalg.norm(init_hand_jerk)))

action1_log.append(np.nan); action2_log.append(np.nan); action_same_log.append(False)

# -------------------------
# 6) シミュレーション実行ループ（best_model を用いて行動を生成）
# -------------------------
MAX_STEPS_RUN = 2000  # 必要に応じて増減

for step_i in range(MAX_STEPS_RUN):
    # model.predict に渡す obs の形状に注意。VecEnv の場合は (n_envs, obs_dim) かもしれないので、
    # ここでは model.predict が期待する形式で obs をそのまま渡す (ユーザー環境次第)。
    action, _ = best_model.predict(obs, deterministic=True)

    # 整形: action が (2,) であることを確認
    action = np.asarray(action).reshape(-1)
    if action.shape[0] != 2:
        raise RuntimeError(f"predict returned action with wrong shape {action.shape}. expected (2,) for 2-joint environment.")

    # env.step 実行
    next_return = env.step(action)

    # Gymnasium と古い gym の返り値形式に対応
    if isinstance(next_return, tuple) and len(next_return) == 5:
        obs, reward, terminated, truncated, info = next_return
        done = bool(terminated or truncated)
    elif isinstance(next_return, tuple) and len(next_return) == 4:
        obs, reward, done, info = next_return
        terminated = bool(done)
        truncated = False
    else:
        raise RuntimeError("env.step() の返り値の形式が不明です。")

    # 時刻進行：base_env に dt があればそれを使い、なければグローバル DT にフォールバック
    dt = getattr(base_env, "dt", None)
    if dt is None:
        try:
            dt = float(DT)
        except Exception:
            dt = 0.01
    t += dt
    time_log.append(t)

    # info 優先で状態を取得。なければ base_env 属性を使う（フォールバック）
    if info is not None:
        # 手先位置
        if "hand_pos" in info:
            hand_pos = np.asarray(info["hand_pos"]).reshape(-1)[:2]
        else:
            # forward_kinematics があればそれを使う
            if hasattr(base_env, "forward_kinematics"):
                try:
                    # theta は info か base_env のどちらかから取る
                    theta_info = np.asarray(info.get("theta", getattr(base_env, "theta", init_theta))).reshape(-1)[:2]
                    hand_pos = base_env.forward_kinematics(theta_info)
                except Exception:
                    hand_pos = np.array([np.nan, np.nan])
            else:
                # 最終フォールバック（1関節的に計算）
                try:
                    L_val = float(getattr(base_env, "l", getattr(base_env, "l1", 1.0) + getattr(base_env, "l2", 0.0)))
                    theta0 = float(np.asarray(info.get("theta", getattr(base_env, "theta", init_theta)))[0])
                    hand_pos = L_val * np.array([np.cos(theta0), np.sin(theta0)])
                except Exception:
                    hand_pos = np.array([np.nan, np.nan])

        theta_arr = np.asarray(info.get("theta", getattr(base_env, "theta", init_theta))).reshape(-1)[:2]
        theta_vel_arr = np.asarray(info.get("theta_vel", getattr(base_env, "theta_vel", init_theta_vel))).reshape(-1)[:2]
        theta_acc_arr = np.asarray(info.get("theta_acc", getattr(base_env, "theta_acc", init_theta_acc))).reshape(-1)[:2]
        theta_jerk_arr = np.asarray(info.get("theta_jerk", getattr(base_env, "theta_jerk", init_theta_jerk))).reshape(-1)[:2]

        # 手先の速度/加速度/躍度が info にあるならそれを使う（推奨）
        hand_vel = np.asarray(info.get("hand_vel", getattr(base_env, "hand_vel", np.zeros(2)))).reshape(-1)[:2]
        hand_acc = np.asarray(info.get("hand_acc", getattr(base_env, "hand_acc", np.zeros(2)))).reshape(-1)[:2]
        hand_jerk = np.asarray(info.get("hand_jerk", getattr(base_env, "hand_jerk", np.zeros(2)))).reshape(-1)[:2]
    else:
        # info が None の場合は base_env の属性を見に行く
        try:
            # base_env.forward_kinematics があるなら theta から手先位置を計算
            theta_arr = np.asarray(getattr(base_env, "theta", init_theta)).reshape(-1)[:2]
            if hasattr(base_env, "forward_kinematics"):
                hand_pos = base_env.forward_kinematics(theta_arr)
            else:
                L_val = float(getattr(base_env, "l", getattr(base_env, "l1", 1.0) + getattr(base_env, "l2", 0.0)))
                hand_pos = L_val * np.array([np.cos(theta_arr[0]), np.sin(theta_arr[0])])
            theta_vel_arr = np.asarray(getattr(base_env, "theta_vel", init_theta_vel)).reshape(-1)[:2]
            theta_acc_arr = np.asarray(getattr(base_env, "theta_acc", init_theta_acc)).reshape(-1)[:2]
            theta_jerk_arr = np.asarray(getattr(base_env, "theta_jerk", init_theta_jerk)).reshape(-1)[:2]
            hand_vel = np.asarray(getattr(base_env, "hand_vel", np.zeros(2))).reshape(-1)[:2]
            hand_acc = np.asarray(getattr(base_env, "hand_acc", np.zeros(2))).reshape(-1)[:2]
            hand_jerk = np.asarray(getattr(base_env, "hand_jerk", np.zeros(2))).reshape(-1)[:2]
        except Exception:
            theta_arr = init_theta
            theta_vel_arr = init_theta_vel
            theta_acc_arr = init_theta_acc
            theta_jerk_arr = init_theta_jerk
            hand_pos = np.array([np.nan, np.nan])
            hand_vel = np.zeros(2)
            hand_acc = np.zeros(2)
            hand_jerk = np.zeros(2)

    # ----- ログに追加 -----
    hand_x.append(float(hand_pos[0])); hand_y.append(float(hand_pos[1]))
    theta1_log.append(np.degrees(theta_arr[0])); theta2_log.append(np.degrees(theta_arr[1]))
    theta1_vel_log.append(np.degrees(theta_vel_arr[0])); theta2_vel_log.append(np.degrees(theta_vel_arr[1]))
    theta1_acc_log.append(np.degrees(theta_acc_arr[0])); theta2_acc_log.append(np.degrees(theta_acc_arr[1]))
    theta1_jerk_log.append(np.degrees(theta_jerk_arr[0])); theta2_jerk_log.append(np.degrees(theta_jerk_arr[1]))

    # アクション記録（deg/s 単位で保存）
    action_deg = np.degrees(action)
    action1_log.append(float(action_deg[0])); action2_log.append(float(action_deg[1]))
    action_same_log.append(bool(np.allclose(action_deg[0], action_deg[1], atol=1e-6)))

    # 手先ノルム（SI 単位）を計算してログ
    hand_speed_log.append(float(np.linalg.norm(hand_vel)))
    hand_acc_norm_log.append(float(np.linalg.norm(hand_acc)))
    hand_jerk_norm_log.append(float(np.linalg.norm(hand_jerk)))

    # 終了判定: terminated/truncated または安全上の上限ステップ数など
    if bool(terminated) or bool(truncated) or (len(time_log) > 100000):
        break

# -------------------------
# 7) DataFrame 作成 & CSV 保存（2関節・手先ノルム列あり）
# -------------------------
df = pd.DataFrame({
    "Time": time_log,
    "HandX": hand_x,
    "HandY": hand_y,
    "HandSpeed (m/s)": hand_speed_log,
    "HandAcc (m/s^2)": hand_acc_norm_log,
    "HandJerk (m/s^3)": hand_jerk_norm_log,
    "Theta1 (deg)": theta1_log,
    "Theta2 (deg)": theta2_log,
    "Theta1_vel (deg/s)": theta1_vel_log,
    "Theta2_vel (deg/s)": theta2_vel_log,
    "Theta1_acc (deg/s^2)": theta1_acc_log,
    "Theta2_acc (deg/s^2)": theta2_acc_log,
    "Theta1_jer (deg/s^3)": theta1_jerk_log,
    "Theta2_jer (deg/s^3)": theta2_jerk_log,
    "Action1 (deg/s)": action1_log,
    "Action2 (deg/s)": action2_log,
    "Action_same": action_same_log
})

output_dir = latest_log_dir
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, "best_result_2joint_with_initial_and_handnorms.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV 保存: {csv_path}")

# -------------------------
# 8) プロット: 手先軌跡、関節ダイナミクス（両関節表示）、手先ノルム
# -------------------------
# ===== プロット: 手先軌道 =====
# ===== プロット: 手先軌道 + アーム姿勢を重ねる =====
fig1, ax1 = plt.subplots(figsize=(6,6))

# shoulder を原点に固定
shoulder = np.array([0.0, 0.0])

# 手先軌道（青ライン）
ax1.plot(df["HandX"], df["HandY"], linestyle="-", linewidth=1.5, label="end-effector trajectory")

# start / end markers
ax1.scatter(df["HandX"].iloc[0], df["HandY"].iloc[0], color="red", label="start", zorder=5)
ax1.scatter(df["HandX"].iloc[-1], df["HandY"].iloc[-1], color="green", label="end", zorder=5)

# リンク長（env から取得できなければ L を半分ずつ使う）
try:
    l1 = float(getattr(env, "l1", (L*0.5)))
    l2 = float(getattr(env, "l2", (L*0.5)))
except Exception:
    l1 = L * 0.5
    l2 = L * 0.5

# 関節位置を計算（theta は deg で保存されている想定）
theta1_rad = np.radians(df["Theta1 (deg)"].values)
theta2_rad = np.radians(df["Theta2 (deg)"].values)

# 第1関節（肘ではなくリンク1の末端）位置
x1 = l1 * np.cos(theta1_rad)
y1 = l1 * np.sin(theta1_rad)
# 手先（リンク2末端）
x2 = x1 + l2 * np.cos(theta1_rad + theta2_rad)
y2 = y1 + l2 * np.sin(theta1_rad + theta2_rad)

# 確認：手先ログと一致するはず
# (もし mismatch があれば警告を出す)
try:
    mismatch = np.nanmax(np.abs(np.column_stack([df["HandX"].values, df["HandY"].values]) - np.column_stack([x2, y2])))
    if mismatch > 1e-6:
        # print 小さめの警告（実行中に表示）
        print(f"[WARN] forward kinematics and logged hand pos mismatch (max diff={mismatch:.6e})")
except Exception:
    pass

# アーム姿勢を複数タイムスタンプで描く
N_POSES = min(15, len(df))  # 最大描画姿勢数
indices = np.linspace(0, len(df)-1, N_POSES).astype(int)

# 薄いグレーで時系列に沿ってアームをプロット（透過）
for idx in indices:
    xs = [0.0, x1[idx], x2[idx]]
    ys = [0.0, y1[idx], y2[idx]]
    ax1.plot(xs, ys, linewidth=1, color=(0.5,0.5,0.5,0.25), solid_capstyle='round')  # 太めの線
    ax1.scatter([xs[1], xs[2]], [ys[1], ys[2]], s=40, edgecolors='k', facecolors=(0.9,0.9,0.9,0.6), zorder=4)

# 強調: start, mid, end の姿勢（色とマーカーで分かりやすく）
# start
i0 = 0
xs0 = [0.0, x1[i0], x2[i0]]
ys0 = [0.0, y1[i0], y2[i0]]
ax1.plot(xs0, ys0, linewidth=2, color='tab:red', solid_capstyle='round', zorder=6)
ax1.scatter([xs0[1], xs0[2]], [ys0[1], ys0[2]], s=80, color='tab:red', edgecolors='k', zorder=7)
ax1.scatter(0.0, 0.0, s=100, color='black', zorder=8)  # shoulder


# end
ie = len(df)-1
xse = [0.0, x1[ie], x2[ie]]
yse = [0.0, y1[ie], y2[ie]]
ax1.plot(xse, yse, linewidth=2, color='tab:green', solid_capstyle='round', zorder=6)
ax1.scatter([xse[1], xse[2]], [yse[1], yse[2]], s=100, color='tab:green', edgecolors='k', zorder=7)

# # 関節角度の数値も近傍に表示（start と end のみ、必要なら増やせます）
# ang_off = 0.03 * (l1 + l2)  # 表示オフセット
# ax1.text(xs0[1]+ang_off, ys0[1]+ang_off, f"{np.degrees(theta1_rad[i0]):.1f}°", color='tab:red', fontsize=9)
# ax1.text(xs0[2]+ang_off, ys0[2]+ang_off, f"{np.degrees(theta2_rad[i0]):.1f}°", color='tab:red', fontsize=9)
# ax1.text(xse[1]+ang_off, yse[1]+ang_off, f"{np.degrees(theta1_rad[ie]):.1f}°", color='tab:green', fontsize=9)
# ax1.text(xse[2]+ang_off, yse[2]+ang_off, f"{np.degrees(theta2_rad[ie]):.1f}°", color='tab:green', fontsize=9)

# 装飾
ax1.set_xlabel("X [m]"); ax1.set_ylabel("Y [m]")
# ax1.set_title("End-effector Trajectory with Arm Poses (shoulder at (0,0))")
ax1.grid(True)
ax1.set_aspect('equal', adjustable='box')

# 自動スケール（アーム長に沿ったマージンを付ける）
reach = l1 + l2
ax1.set_xlim(-reach-0.1*reach, reach+0.1*reach)
ax1.set_ylim(-0.1*reach, reach+0.6*reach)  # y 上方向に少し余裕

plt.tight_layout()
plt.savefig(f"{base_filename}_trajectory_with_arms.png", dpi=300)
plt.show()

# 関節動的変数（Theta, Vel, Acc, Jerk）: 各プロットに Joint1/Joint2 を表示
fig2, axes = plt.subplots(4, 1, figsize=(8, 12), sharex=True)
axes[0].plot(df["Time"], df["Theta1 (deg)"], label="Theta1"); axes[0].plot(df["Time"], df["Theta2 (deg)"], linestyle="--", label="Theta2")
axes[0].set_ylabel("Theta (deg)"); axes[0].legend(); axes[0].grid()
axes[0].set_ylim(0, 180)

axes[1].plot(df["Time"], df["Theta1_vel (deg/s)"], label="Vel1"); axes[1].plot(df["Time"], df["Theta2_vel (deg/s)"], linestyle="--", label="Vel2")
axes[1].set_ylabel("Velocity (deg/s)"); axes[1].legend(); axes[1].grid()

axes[2].plot(df["Time"], df["Theta1_acc (deg/s^2)"], label="Acc1"); axes[2].plot(df["Time"], df["Theta2_acc (deg/s^2)"], linestyle="--", label="Acc2")
axes[2].set_ylabel("Acceleration (deg/s^2)"); axes[2].legend(); axes[2].grid()

axes[3].plot(df["Time"], df["Theta1_jer (deg/s^3)"], label="Jerk1"); axes[3].plot(df["Time"], df["Theta2_jer (deg/s^3)"], linestyle="--", label="Jerk2")
axes[3].set_ylabel("Jerk (deg/s^3)"); axes[3].set_xlabel("Time (s)"); axes[3].legend(); axes[3].grid()

png_dynamics = os.path.join(output_dir, "best_result_2joint_joint_dynamics.png")
plt.tight_layout(); plt.savefig(png_dynamics, dpi=300); plt.show()
print(f"✅ 関節動的変数プロット保存: {png_dynamics}")

# 手先ノルム（速度・加速度・躍度）
fig3, ax3 = plt.subplots(3,1, figsize=(8,9), sharex=True)
ax3[0].plot(df["Time"], df["HandSpeed (m/s)"], label="||v||"); ax3[0].set_ylabel("Speed (m/s)"); ax3[0].legend(); ax3[0].grid()
ax3[1].plot(df["Time"], df["HandAcc (m/s^2)"], label="||a||"); ax3[1].set_ylabel("Acc (m/s^2)"); ax3[1].legend(); ax3[1].grid()
ax3[2].plot(df["Time"], df["HandJerk (m/s^3)"], label="||jerk||"); ax3[2].set_ylabel("Jerk (m/s^3)"); ax3[2].set_xlabel("Time (s)"); ax3[2].legend(); ax3[2].grid()

png_handnorms = os.path.join(output_dir, "best_result_2joint_hand_norms.png")
plt.tight_layout(); plt.savefig(png_handnorms, dpi=300); plt.show()
print(f"✅ 手先ノルムプロット保存: {png_handnorms}")

# 補助: action の遷移と same フラグ
fig4, ax4 = plt.subplots(2,1, figsize=(8,6), sharex=True)
ax4[0].plot(df["Time"], df["Action1 (deg/s)"], label="Action1"); ax4[0].plot(df["Time"], df["Action2 (deg/s)"], linestyle="--", label="Action2")
ax4[0].set_ylabel("Action (deg/s)"); ax4[0].legend(); ax4[0].grid()
ax4[1].plot(df["Time"], df["Action_same"].astype(int), label="Action_same")
ax4[1].set_ylabel("Action_same"); ax4[1].set_xlabel("Time (s)"); ax4[1].grid()
png_actions = os.path.join(output_dir, "best_result_2joint_actions.png")
plt.tight_layout(); plt.savefig(png_actions, dpi=300); plt.show()
print(f"✅ action プロット保存: {png_actions}")

# ===== 完了メッセージ =====
print("=== 実行完了: 2関節対応の評価ログ & プロットを保存しました ===")
print(f"出力フォルダ: {output_dir}")


## エピソードごとの様々な報酬の推移などの多指標をプロット

In [ ]:
# ====== episode_full_metrics.csv を読み込んで多指標プロットを自動生成するスクリプト ======
# 最新ログフォルダ検出 -> CSV 読込 -> グループ別プロットと個別プロットを保存。
# 追加機能: 各グループプロットについて、移動平均(rolling mean)版も出力。

plt.rcParams["font.size"] = 12

# ===== 設定: 移動平均窓幅 =====
MOVING_AVG_WINDOW = 10  # ここを変えれば移動平均の平滑化窓を調整できます（エピソード数に応じて適宜設定）

# ===== 最新ログフォルダを取得 =====
logs_dir = "./logs"
log_folders = [os.path.join(logs_dir, d) for d in os.listdir(logs_dir) if os.path.isdir(os.path.join(logs_dir, d))]
if not log_folders:
    raise FileNotFoundError("`./logs` 内にフォルダが見つかりません。学習ログがあるディレクトリを確認してください。")
latest_log_folder = max(log_folders, key=os.path.getmtime)
print(f"[INFO] 最新ログフォルダ: {latest_log_folder}")

# ===== CSV パス =====
csv_filename = "episode_full_metrics.csv"
csv_path = os.path.join(latest_log_folder, csv_filename)
if not os.path.exists(csv_path):
    # 場合によってはファイル名が違う可能性があるので近い名前を探す
    candidates = glob.glob(os.path.join(latest_log_folder, "*episode*.csv")) + glob.glob(os.path.join(latest_log_folder, "*full*.csv"))
    if candidates:
        csv_path = candidates[-1]
        print(f"[WARN] 指定ファイルが見つかりませんでした。代わりに候補を使用します: {csv_path}")
    else:
        raise FileNotFoundError(f"{csv_filename} が見つかりません: {latest_log_folder}")

# csv_path ="logs/2025_10_21_22_45_48/episode_full_metrics.csv"
# ===== CSV 読み込み（ヘッダが1行目にあることを想定） =====
try:
    df = pd.read_csv(csv_path)
    print(f"[INFO] CSVを読み込みました: {csv_path}")
except Exception as e:
    # 場合によっては1行目がコメントなので skiprows=1 を試す
    print("[WARN] 直接読み込みに失敗しました。skiprows=1 を試します。エラー:", e)
    df = pd.read_csv(csv_path, skiprows=1)
    print(f"[INFO] CSVを読み込みました（skiprows=1）: {csv_path}")

# ===== 列名と表示ラベル・単位のマッピング =====
label_unit_map = {
    "episode":                ("Episode", ""),
    "episode_length":         ("Episode length", "steps"),
    "total_reward":           ("Total reward", "arb."),    # 任意単位（報酬は設計次第）
    "sum_reward_dist_step":   ("Sum distance shaping reward", "arb."),
    "sum_reward_jerk_step":   ("Sum jerk shaping reward", "arb."),
    "sum_reward_time_step":   ("Sum time shaping reward", "arb."),
    "sum_terminal_jerk_penalty": ("Terminal jerk penalty", "arb."),
    "sum_terminal_vel_penalty":  ("Terminal velocity penalty", "arb."),
    "sum_time_bonus":         ("Terminal time bonus", "arb."),
    "jerk_sum":               ("Jerk sum", "rad^2/s^5"),
    "success":                ("Success", "0/1"),
    "actor_loss":             ("Actor loss", "arb."),
    "critic_loss":            ("Critic loss", "arb."),
    "ent_coef":               ("Entropy coeff", "arb."),
    "ent_coef_loss":          ("Ent coef loss", "arb."),
    "avg_q":                  ("Average Q", "arb."),
    "running_mean_total_reward_100": ("Running mean total reward (100)", "arb."),
    "running_success_rate_100": ("Running success rate (100)", "ratio"),
    "episode_wall_time":      ("Episode wall time", "s"),
    "sigma_T":  ("sigma_T", ""),
}

# ===== CSV 中の列名（利用可能なもの）を表示 =====
print("[INFO] CSV の列:")
for c in df.columns:
    print("  ", c)

# ===== 基本 X 軸: エピソード番号取得 =====
if "episode" in df.columns:
    episodes = df["episode"].values
else:
    episodes = np.arange(1, len(df) + 1)
    print("[WARN] 'episode' 列が見つかりません。インデックスをエピソード番号として使用します。")

# ===== グループ定義：同じ図にまとめたい列をグループ化 =====
groups = {
    "reward_components": [
        "total_reward",
        "sum_terminal_jerk_penalty",
        "sum_terminal_vel_penalty",
        "sum_time_bonus",
        "sum_reward_dist_step",
        "sum_reward_jerk_step",
        "sum_reward_time_step",
    ],
    "reward_goal_components": [
        "sum_terminal_jerk_penalty",
        "sum_terminal_vel_penalty",
        "sum_time_bonus"
    ],
    "reward__step_components": [
        "sum_reward_dist_step",
        "sum_reward_jerk_step",
        "sum_reward_time_step",
    ],
    "dynamics": [
        "jerk_sum",
        "episode_length",
        "episode_wall_time"
    ],
    "learning_metrics": [
        "actor_loss",
        "critic_loss",
        "ent_coef",
        "ent_coef_loss",
        "avg_q"
    ],
    "running_stats": [
        "running_mean_total_reward_100",
        "running_success_rate_100",
        "success"
    ]
}

# ===== 保存ディレクトリ（最新ログフォルダ内に出す） =====
out_dir = latest_log_folder
plots_dir = os.path.join(out_dir, "plots")
os.makedirs(plots_dir, exist_ok=True)

# ===== 描画ヘルパー関数 =====
def safe_get_col(df, col):
    return df[col].values if col in df.columns else None

def plot_series(x, y,xlabel,ylabel, savepath, legend=None, ylim=None):
    """単一系列または複数系列を描画して保存するヘルパー"""
    if y is None:
        print(f"[SKIP] {title} : データが存在しません。")
        return
    plt.figure(figsize=(10, 4.5))
    if isinstance(y, dict):
        # 複数系列
        for name, arr in y.items():
            plt.plot(x, arr, label=name)
    else:
        plt.plot(x, y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if legend is not None:
        plt.legend(loc="best")
    plt.grid(True)
    if ylim is not None:
        plt.ylim(ylim)
    plt.tight_layout()
    plt.savefig(savepath, dpi=300)
    plt.close()
    print(f"[SAVED] {savepath}")

# ===== 便利: 移動平均を作る関数 =====
def moving_average(arr, window):
    """NaN を許容しつつ単純移動平均を返す。pandas を使って簡潔に処理。"""
    if arr is None:
        return None
    # pandas.Series を使って rolling(mean) を計算（min_periods=1 で立ち上がりも値を返す）
    return pd.Series(arr).rolling(window=window, min_periods=1).mean().to_numpy()

# ===== 1) グループごとのまとめプロット（同じスケールで重ねて見たいもの） =====
for gname, cols in groups.items():
    # 実際に存在する列だけ取り出す
    available = [c for c in cols if c in df.columns]
    if not available:
        print(f"[SKIP] グループ '{gname}' にプロット可能な列がありません。")
        continue

    # --- 元データでのプロット（元からある動作） ---
    series_dict = {}
    legend_labels = []
    for c in available:
        arr = df[c].values
        label, unit = label_unit_map.get(c, (c, ""))
        label_full = f"{label} [{unit}]" if unit else label
        series_dict[label_full] = arr
        legend_labels.append(label_full)

    title = f"{gname} (group) over episodes"
    savepath = os.path.join(plots_dir, f"group_{gname}.png")
    plot_series(episodes, series_dict, "Episode", "Value", savepath, legend=True)

    # --- 追加: 移動平均版のプロット（同じ形式） ---
    # 各系列に対して移動平均を取り、新しい辞書を作る
    ma_series_dict = {}
    for c in available:
        arr = df[c].values
        arr_ma = moving_average(arr, MOVING_AVG_WINDOW)
        label, unit = label_unit_map.get(c, (c, ""))
        label_full = f"{label} [{unit}]" if unit else label
        ma_series_dict[label_full] = arr_ma

    title_ma = f"{gname} (group) - moving avg (window={MOVING_AVG_WINDOW})"
    savepath_ma = os.path.join(plots_dir, f"group_{gname}_ma_w{MOVING_AVG_WINDOW}.png")
    plot_series(episodes, ma_series_dict, "Episode", f"Mean Value (window={MOVING_AVG_WINDOW})", savepath_ma, legend=None)

# ===== 2) 各列ごとの個別プロット（ラベルと単位を軸に反映） =====
for col in df.columns:
    if col == "episode":
        continue
    arr = safe_get_col(df, col)
    if arr is None:
        continue

    label, unit = label_unit_map.get(col, (col, ""))
    ylabel = f"{label} [{unit}]" if unit else label
    title = f"{label} over episodes"
    savepath = os.path.join(plots_dir, f"episode_{col}.png")

    # success / running_success_rate の場合、0-1 の比率なので ylim を合せる
    ylim = None
    if col in ("success", "running_success_rate_100"):
        ylim = (-0.05, 1.05)

    plot_series(episodes, arr,"Episode", ylabel, savepath, legend=None, ylim=ylim)

    # 追加: 個別プロットの移動平均（必要なら出力。元データがある場合のみ）
    arr_ma = moving_average(arr, MOVING_AVG_WINDOW)
    savepath_ma = os.path.join(plots_dir, f"episode_{col}_ma_w{MOVING_AVG_WINDOW}.png")
    title_ma = f"{label} - moving avg (window={MOVING_AVG_WINDOW})"
    plot_series(episodes, arr_ma,"Episode", f"{label} (moving avg)", savepath_ma, legend=None, ylim=ylim)

# ===== 3) 全報酬成分だけを重ねたプロット（見やすく色分け） =====
reward_cols = [c for c in groups["reward_components"] if c in df.columns]
if reward_cols:
    series = {}
    for c in reward_cols:
        label, unit = label_unit_map.get(c, (c, ""))
        series[f"{label} [{unit}]"] = df[c].values
    savepath = os.path.join(plots_dir, "all_reward_components_overlay.png")
    plot_series(episodes, series,  "Episode", "Reward (arb.)", savepath, legend=None)

    # 全報酬成分の移動平均オーバーレイ
    series_ma = {}
    for c in reward_cols:
        label, unit = label_unit_map.get(c, (c, ""))
        series_ma[f"{label} [{unit}]"] = moving_average(df[c].values, MOVING_AVG_WINDOW)
    savepath_ma = os.path.join(plots_dir, f"all_reward_components_overlay_ma_w{MOVING_AVG_WINDOW}.png")
    plot_series(episodes, series_ma, "Episode", "Reward (moving avg)", savepath_ma, legend=True)

print("[DONE] プロット作成完了。結果はフォルダに保存されています:", plots_dir)
